This notebook tests the predictions on the training set. The model achieves 100% accuracy quite easily, after a few epochs

In [26]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
import htkmfc
import os
import lasagne
import time
import theano
import theano.tensor as T
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from IPython import display


Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled)


In [10]:
import htkmfc
import os
import numpy as np

#load data to the CPU
def DataLoad():
    
    f1 = open('/misc/data15/reco/bhattgau/Rnn/Lists/spk_softmax/Train_feats_labs.plst')
    lines = f1.readlines()
    lines = [l.strip() for l in lines]
    labelz = [int(l.split()[1]) for l in lines] 
    #labelz = labelz[:20]
    features = [l.split()[0] for l in lines] 
    
    f2 = open('/misc/data15/reco/bhattgau/Rnn/Lists/spk_softmax/Valid_feats_labs.plst')
    lines = f2.readlines()
    lines = [l.strip() for l in lines]
    val_labelz = [int(l.split()[1]) for l in lines] 
    #val_labelz = val_labelz[:20]
    val_features = [l.split()[0] for l in lines] 
    
    n_samp = len(features)
    maxlen=600 #pad all utterances to this length
    feat_dim=20
    nSpk = 98
    dpth = '/misc/data15/reco/bhattgau/Rnn/Data/mfcc/Nobackup/VQ_VAD_HO_EPD/'

    Data = np.zeros((n_samp, maxlen, feat_dim), dtype='float32')
    Mask = np.zeros((n_samp,maxlen), dtype='float32')
    #Targets = np.zeros((n_samp, nSpk), dtype='int32')
    
    vn_samp = len(val_features)
    val_Data = np.zeros((vn_samp, maxlen, feat_dim), dtype='float32')
    val_Mask = np.zeros((vn_samp,maxlen), dtype='float32')
    #Targets = np.zeros((n_samp, nSpk), dtype='int32')

    for ind,f in enumerate(features):
        fname = os.path.join(dpth,f+'.fea')
        fi = htkmfc.HTKFeat_read(fname)
        data = fi.getall()[:,:20]
        Mask[ind,:data.shape[0]] = 1.0
        pad = maxlen - data.shape[0]
        data = np.vstack((data, np.zeros((pad,20), dtype='float32')))
        Data[ind,:,:] = data
        

    for ind,f in enumerate(val_features):
        fname = os.path.join(dpth,f+'.fea')
        fi = htkmfc.HTKFeat_read(fname)
        data = fi.getall()[:,:20]
        val_Mask[ind,:data.shape[0]] = 1.0
        pad = maxlen - data.shape[0]
        data = np.vstack((data, np.zeros((pad,20), dtype='float32')))
        val_Data[ind,:,:] = data


    return Data, Mask, np.asarray(labelz, dtype='int32'), val_Data, val_Mask, np.asarray(val_labelz, dtype='int32')

#Generator that loads minibatches to the GPU

def MbGenerator(inputs, mask, targets, batchsize, shuffle=True):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], mask[excerpt], targets[excerpt]

In [11]:
t_data, t_mask, t_labels, _,_,_ = DataLoad()

In [4]:
import htkmfc
import os
import lasagne
import time
import theano
import theano.tensor as T
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import cPickle

# Min/max sequence length
MAX_LENGTH = 800
# Number of units in the hidden (recurrent) layer
N_HIDDEN = 600
F_DIM = 20
# Number of training sequences ain each batch
N_BATCH = 1
# Optimization learning rate
LEARNING_RATE = .001
# All gradients above this will be clipped
GRAD_CLIP = 100
# How often should we check the output?
EPOCH_SIZE = 100
# Number of epochs to train the net
NUM_EPOCHS = 10

nSpk = 98

X = T.tensor3(name='input',dtype='float32')
Mask = T.matrix(name = 'mask', dtype='float32')

target = T.matrix(name='target_values', dtype='float32')

print("Building network ...")

l_in = lasagne.layers.InputLayer(shape=(None, None, F_DIM), input_var = X)
n_batch,maxlen,_ = l_in.input_var.shape

#get the batch size for the weights matrix
l_mask = lasagne.layers.InputLayer(shape=(None, None), input_var = Mask)
#print lasagne.layers.get_output(l_mask, inputs={l_mask: Mask}).eval({Mask: mask}).shape
#initialize the gates

#Compute Recurrent Embeddings
l_forward = lasagne.layers.GRULayer(l_in, N_HIDDEN, mask_input=l_mask)
l_backward = lasagne.layers.GRULayer(l_in, N_HIDDEN, mask_input=l_mask, backwards=True)
l_sum = lasagne.layers.ElemwiseSumLayer([l_forward, l_backward])

Recc_emb =  lasagne.layers.get_output(l_sum, inputs={l_in: X, l_mask: Mask})#.eval({X: x_dummy, Mask: mask})
#print lasagne.layers.get_output(l_res, inputs={l_in: X, l_mask: Mask}).eval({X: x_dummy, Mask: mask}).shape

l_reshape1 = lasagne.layers.ReshapeLayer(l_sum, (-1, N_HIDDEN))

l_attend = lasagne.layers.DenseLayer(l_reshape1, num_units=1, nonlinearity=lasagne.nonlinearities.sigmoid)
#print lasagne.layers.get_output(l_attend, inputs={l_in: X, l_mask: Mask}).eval({X: x_dummy, Mask: mask}).shape

l_attention = lasagne.layers.ReshapeLayer(l_attend, (n_batch, maxlen))
attn_wts = lasagne.layers.get_output(l_attention, inputs={l_in: X, l_mask: Mask})#.eval({X: x_dummy, Mask: mask})
attn_probs = T.nnet.softmax(attn_wts)#.eval({X:x_dummy, Mask:mask})

get_wts = theano.function([X,Mask], attn_probs)

#Calculate the last time-step of a recording using the mask.
lstep = T.sum(Mask, axis=1)#.eval({Mask : mask})
lstep = T.cast(lstep, 'int32')#.eval()
lstep = lstep.T

#print lasagne.layers.get_output(l_reshape1b, inputs={l_in: X, l_mask: Mask}).eval({X: x_dummy, Mask: mask}).shape
Wts = T.zeros([n_batch, maxlen], dtype='float32')

def weighted_avg(w_t, l_t, W, R_emb):
    W = T.set_subtensor(W[:l_t], w_t[:l_t])
    W = W[None,:]
    w_sum = T.dot(W, R_emb)
    
    #this W (1,600) is is then multiplied (dot product) with the recurrent embedding (600, 100)
    #to produce a feature vector that is a weighted sum of all the embedding vectors of the recording
    #also return the weights for analysis
    
    return w_sum
    
U_t,_ = theano.scan(fn=weighted_avg, sequences=[attn_probs, lstep, Wts, Recc_emb])
#extract_Rvector =U_t.eval({X:x_dummy,Mask:mask})

l_in2 = lasagne.layers.InputLayer(shape=(None,None, None), input_var = U_t)
n_batch1,l1,hdim = l_in2.input_var.shape

l_reshape2 = lasagne.layers.ReshapeLayer(l_in2, (n_batch1*l1,N_HIDDEN))
#print lasagne.layers.get_output(l_reshape2, inputs={l_in: X, l_mask: Mask, l_in2: U_t}).eval({X: x_dummy, Mask: mask}).shape

#Finally this feature vector(s) gets passed to a dense layer which represents a softmax distribution over speakers
l_Spk_softmax = lasagne.layers.DenseLayer(l_reshape2, num_units=98, nonlinearity=lasagne.nonlinearities.softmax)

Building network ...


In [16]:
#load trained model
f_load = open('/misc/data15/reco/bhattgau/Rnn/Projects/Rvector/Weights/basic-softmax/Attn2_softmax_600.pkl')
f_load1 = open('/misc/data15/reco/bhattgau/Rnn/Projects/Rvector/Weights/basic-softmax/Attn2_softmax_600b.pkl')

Wts = cPickle.load(f_load)
Wts1 = cPickle.load(f_load1)

lasagne.layers.set_all_param_values(l_attention, Wts)
lasagne.layers.set_all_param_values(l_Spk_softmax, Wts1)


#Prediction = T.argmax(network_output, axis=1)#.eval({X: t_data, Mask: t_mask})

In [17]:
wts = get_wts(t_data[:10],t_mask[:10])

In [20]:
wts[0]

array([ 0.00300758,  0.00300759,  0.0030076 ,  0.00300759,  0.00300755,
        0.00300758,  0.00300756,  0.00300751,  0.00300756,  0.00300759,
        0.00300759,  0.00300759,  0.00300759,  0.00300759,  0.00300756,
        0.0030075 ,  0.00300757,  0.00300759,  0.00300759,  0.00300758,
        0.00300759,  0.00300758,  0.00300759,  0.00300759,  0.00300759,
        0.00300758,  0.00300759,  0.00300759,  0.00300755,  0.00300758,
        0.00300757,  0.00300757,  0.00300756,  0.00300757,  0.00300759,
        0.0030076 ,  0.0030076 ,  0.00300756,  0.00300759,  0.00300759,
        0.00300759,  0.00300758,  0.00300752,  0.0030075 ,  0.00300756,
        0.00300758,  0.00300759,  0.00300759,  0.00300759,  0.00300759,
        0.00300757,  0.00300745,  0.00299908,  0.00300267,  0.0030067 ,
        0.00300734,  0.00300753,  0.00300759,  0.00300759,  0.0030076 ,
        0.0030076 ,  0.0030076 ,  0.00300759,  0.00300757,  0.0030075 ,
        0.00300723,  0.00300729,  0.00300754,  0.00300757,  0.00

In [4]:
#Extract R-vectors
def iterate_minibatches(inputs, mask, batchsize, shuffle=True):
    iplen = len(inputs)
    pointer = 0
    indices=np.arange(iplen)
    
    while pointer < iplen:
    
        if pointer <= iplen - batchsize:
            excerpt = indices[pointer:pointer+batchsize]
        else:
            batchsize = iplen-pointer
            excerpt = indices[pointer:pointer+batchsize]
        
        pointer+=batchsize

        yield inputs[excerpt], mask[excerpt]

In [5]:
Rvectors = lasagne.layers.get_output(l_in2)#.eval({X: t_data, Mask: t_mask})

In [ ]:
#Score Model
from scipy.spatial.distance import cosine
import sidekit

f1 = open('/misc/data15/reco/bhattgau/Rnn/Lists/spk_softmax/Train_feats_labs.plst')
lines = f1.readlines()
lines = [l.strip() for l in lines]
labelz = [int(l.split()[1]) for l in lines] 
#labelz = labelz[:20]
features = [l.split()[0] for l in lines]

spkID = [f.split('_')[0] for f in features]

labels=[]

for sp in spkID:
    for sp1 in spkID:
        if sp==sp1:
            labels.append(1)
        else:
            labels.append(0)

scores=[]

for r in Rvector_trn:
    for r1 in Rvector_trn:
        scores.append(1-cosine(r,r1))

        
tar_scores = []
non_scores = []

for l,s in zip(labels, scores):
    if l==1:
        tar_scores.append(s)
    else:
        non_scores.append(s)
tar = np.asarray(tar_scores, dtype='float32')
non = np.asarray(non_scores, dtype='float32')

_,_,eer,_ = sidekit.bosaris.detplot.rocchdet(tar, non)
eer

The error rate on the dev set is 0.3, now we try on the test set. 

In [6]:
#process enrollment speakers
fen = open('/misc/data15/reco/bhattgau/Rnn/Lists/test_lists/enroll.plst')
lines = fen.readlines()
lines = [l.strip() for l in lines]
en1 = [l.split()[1] for l in lines]
en2 = [l.split()[2] for l in lines]
en3 = [l.split()[3] for l in lines]
enroll = en1+en2+en3
enroll = np.sort(enroll)

enrID = [e.split('_')[0] for e in enroll]
enrIDs = np.sort(list(set(enrID)))

In [22]:
enroll[:13]

array(['2002_0101_1031_V_00_E_D_79f7_0915102653_0_AndroidAPP',
       '2002_0101_1031_V_00_E_D_79f7_0915102715_1_AndroidAPP',
       '2002_0101_1031_V_00_E_D_79f7_0915102733_2_AndroidAPP',
       '2002_0101_1031_V_00_E_L_4eae_0611143410_0_L',
       '2002_0101_1031_V_00_E_L_4eae_0611143421_1_L',
       '2002_0101_1031_V_00_E_L_4eae_0611143432_2_L',
       '2002_0101_1031_V_00_E_M_53cd_0611144155_0_SAM',
       '2002_0101_1031_V_00_E_M_53cd_0611144206_1_SAM',
       '2002_0101_1031_V_00_E_M_53cd_0611144218_2_SAM',
       '2005_0101_1031_V_00_E_D_6241_0911133057_0_AndroidAPP',
       '2005_0101_1031_V_00_E_D_6241_0911133210_1_AndroidAPP',
       '2005_0101_1031_V_00_E_D_6241_0911133225_2_AndroidAPP',
       '2005_0101_1031_V_00_E_L_3aed_0617075626_0_L'], 
      dtype='|S55')

In [23]:
dpth = '/misc/data15/reco/bhattgau/Rnn/Data/mfcc/Nobackup/VQ_VAD_HO_EPD/'
maxlen = 800
f_dim = 20
R_VEC=[]

n_samp = len(enroll)

E_data=np.zeros((n_samp,maxlen,f_dim), dtype='float32')
E_mask=np.zeros((n_samp,maxlen), dtype='float32')

ptr=0

for ids in enrIDs:
    enr_sess = [s for s in enroll if ids in s.split('_')[0]]
    ses_len = len(enr_sess)
    EData = np.zeros((ses_len,maxlen,f_dim), dtype='float32')
    EMask = np.zeros((ses_len,maxlen), dtype='float32')
    
    for ind,sess in enumerate(enr_sess):
        print sess


    '''
    for ind,sess in enumerate(enr_sess):
        
        fname = os.path.join(dpth,sess+'.fea')
        fi = htkmfc.HTKFeat_read(fname)
        data = fi.getall()[:,:20]
        EMask[ind,:data.shape[0]] = 1.0
        pad = maxlen - data.shape[0]
        data = np.vstack((data, np.zeros((pad,20), dtype='float32')))
        EData[ind,:,:] = data
    
    Nsamp = EData.shape[0]
    E_data[ptr:ptr+Nsamp,:,:] = EData  
    E_mask[ptr:ptr+Nsamp,:] = EMask
    ptr+=Nsamp
    
    #extract the set of R-vectors
    #Rvector_enr = lasagne.layers.get_output(l_concat).eval({X: EData, Mask: EMask})
        
    #Rvec_avg = np.mean(Rvector_enr, axis=0)
    #R_VEC.append(Rvec_avg)
    '''

2002_0101_1031_V_00_E_D_79f7_0915102653_0_AndroidAPP
2002_0101_1031_V_00_E_D_79f7_0915102715_1_AndroidAPP
2002_0101_1031_V_00_E_D_79f7_0915102733_2_AndroidAPP
2002_0101_1031_V_00_E_L_4eae_0611143410_0_L
2002_0101_1031_V_00_E_L_4eae_0611143421_1_L
2002_0101_1031_V_00_E_L_4eae_0611143432_2_L
2002_0101_1031_V_00_E_M_53cd_0611144155_0_SAM
2002_0101_1031_V_00_E_M_53cd_0611144206_1_SAM
2002_0101_1031_V_00_E_M_53cd_0611144218_2_SAM
2005_0101_1031_V_00_E_D_6241_0911133057_0_AndroidAPP
2005_0101_1031_V_00_E_D_6241_0911133210_1_AndroidAPP
2005_0101_1031_V_00_E_D_6241_0911133225_2_AndroidAPP
2005_0101_1031_V_00_E_L_3aed_0617075626_0_L
2005_0101_1031_V_00_E_L_3aed_0617075644_1_L
2005_0101_1031_V_00_E_L_3aed_0617075658_2_L
2005_0101_1031_V_00_E_L_a853_0610140024_0_L
2005_0101_1031_V_00_E_L_a853_0610140035_1_L
2005_0101_1031_V_00_E_L_a853_0610140046_2_L
2005_0101_1031_V_00_E_M_e89a_0610140656_0_IPH
2005_0101_1031_V_00_E_M_e89a_0610140707_1_IPH
2005_0101_1031_V_00_E_M_e89a_0610140718_2_IPH
2007_0101_

In [11]:
#Store the Rvectors of the enrollment utterances sorted by speaker id
f_dim = 600
R_vecs = np.zeros((len(enroll), f_dim), dtype='float32')
ptr=0
for batch in iterate_minibatches(E_data, E_mask, 512):
    data, mask  = batch
    Rvector_en = lasagne.layers.get_output(l_in2).eval({X: data, Mask:mask})
    Rvector_en = np.reshape(Rvector_en,(Rvector_en.shape[0], Rvector_en.shape[2]))
    nsamp = Rvector_en.shape[0]
    R_vecs[ptr:ptr+nsamp,:] = Rvector_en
    ptr = ptr + nsamp

In [14]:
#determine which Rvectors belong to a specific speaker and find calculate the average model 
ptr=0
lens=[]
ENR_MODELS=[]
for ids in enrIDs:
    enr_sess = [s for s in enroll if ids in s.split('_')[0]]
    slen = len(enr_sess)
    spk_sess = R_vecs[ptr:ptr+slen,:]
    spk_model = np.mean(R_vecs, axis=0)
    ENR_MODELS.append(spk_model)

In [15]:
#Extract test Rvectors
ftst = open('/misc/data15/reco/bhattgau/Rnn/Lists/test_lists/test_utt.plst')
dpth = '/misc/data15/reco/bhattgau/Rnn/Data/mfcc/Nobackup/VQ_VAD_HO_EPD/'

maxlen=800
fdim=20

lines = ftst.readlines()
lines = [l.strip() for l in lines]
lines = [l.split()[0] for l in lines]
tst_ids = [l.split('_')[0] for l in lines]

In [16]:
nSamp = len(lines)


tst_data = np.zeros((nSamp,maxlen,fdim), dtype='float32')
tst_mask = np.zeros((nSamp,maxlen), dtype='float32')

for ind,f in enumerate(lines):
    fname = os.path.join(dpth,f+'.fea')
    fi = htkmfc.HTKFeat_read(fname)
    data = fi.getall()[:,:20]
    tst_mask[ind,:data.shape[0]] = 1.0
    pad = maxlen - data.shape[0]
    data = np.vstack((data, np.zeros((pad,20), dtype='float32')))
    tst_data[ind,:,:] = data

In [17]:
f_dim = 600
R_vecs_tst = np.zeros((len(lines), f_dim), dtype='float32')
ptr=0
for batch in iterate_minibatches(tst_data, tst_mask, 512):
    data, mask  = batch
    Rvector_tst = lasagne.layers.get_output(l_in2).eval({X: data, Mask:mask})
    Rvector_tst = np.reshape(Rvector_tst,(Rvector_tst.shape[0], Rvector_tst.shape[2]))
    nsamp = Rvector_tst.shape[0]
    R_vecs_tst[ptr:ptr+nsamp,:] = Rvector_tst
    ptr = ptr + nsamp

In [18]:
from scipy.spatial.distance import cosine
import sidekit

#Evaluate EER
#Determine labels
labels=[]

for eids in enrIDs:
    for tids in tst_ids:
        if eids == tids:
            labels.append(1)
        else:
            labels.append(0)

scores=[]

for e in ENR_MODELS:
    for r in Rvector_tst:
        scores.append(1-cosine(e,r))

        
tar_scores = []
non_scores = []

for l,s in zip(labels, scores):
    if l==1:
        tar_scores.append(s)
    else:
        non_scores.append(s)
tar = np.asarray(tar_scores, dtype='float32')
non = np.asarray(non_scores, dtype='float32')

_,_,eer,_ = sidekit.bosaris.detplot.rocchdet(tar, non)
eer

/misc/home/reco/bhattaga/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1318: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


0.49652384634360214

In [20]:
non[:100]

array([ 0.35253274,  0.3868731 ,  0.25732431,  0.34983101,  0.32816324,
        0.35139039,  0.23796652,  0.23399529,  0.27300853,  0.22247155,
        0.22602905,  0.36260307,  0.29717413,  0.21604861,  0.1944629 ,
        0.28000811,  0.26533121,  0.22377397,  0.32302523,  0.19722244,
        0.20255733,  0.17850703,  0.30722067,  0.32948127,  0.25398922,
        0.25867102,  0.29023737,  0.30790275,  0.33102581,  0.29066333,
        0.32257682,  0.33487213,  0.32205695,  0.32547769,  0.27621916,
        0.29515269,  0.3040784 ,  0.33123472,  0.24642   ,  0.18774579,
        0.20201449,  0.25433803,  0.24543706,  0.32046324,  0.25632766,
        0.14256215,  0.22748737,  0.31706044,  0.20735765,  0.26286712,
        0.21196283,  0.34726131,  0.33168733,  0.32554358,  0.32738793,
        0.35087779,  0.34063438,  0.40230164,  0.41625732,  0.26818249,
        0.2385013 ,  0.33964875,  0.32390323,  0.26875713,  0.37176958,
        0.29776457,  0.29118621,  0.38884413,  0.31349844,  0.30

In [25]:
fen = open('/misc/data15/reco/bhattgau/Rnn/Lists/test_lists/enroll.plst')
lines = fen.readlines()
lines = [l.strip() for l in lines]
en0 = [l.split()[0] for l in lines]
en0 = [l.split('_')[0] for l in lines]

In [33]:
R=[]
for i in range(0,R_vecs.shape[0]-1,3):
    indices = np.arange(R_vecs.shape[0])
    excerpt = indices[i:i+2] 
    avg = np.mean(R_vecs[excerpt], axis=0)
    R.append(avg)